# Polynomial Resampling

## Exploring algorithms for real-time resampling/interpolation of sensor-data.

As opposed to batchwise resampling, where you have the entire time-series with many datapoints available to recreate the signal, we are assuming that we only have a short sliding window of datapoint. This is a result of having the resampling algorithm be highly performant, and also of having to resample in real-time on an incoming stream of data.

We are also assuming that the incoming data is non-uniform in the sense that the data is not coming at specified regular interval. Some of the data might also be missing at the time when resampling has to be performed.

This document will explore some of the possibilities and options for resampling by fitting the datapoints with piecewise polynomial interpolation functions.

### Choosing the Polynomial Degree

A linear fit between datapoints is the simplest solution to implement. And one that requires the least amount of data to perform. All that is needed are two points to interpolate between. As the degree of the polynomial increases more data is required, as the polynomial has free parameters that must be determined. But, higher polynomial degree also yields better fitting curves with smoother transitions between segments of the piecewise interpolation.

We argue that cubic polynomials are the best choice as this is the smallest polynomial degree which allows one to not only align the points, but also align the first derivates resulting in smooth transisition from one polynomial piece to another.

In [ ]:
import numpy as np
from bokeh.plotting import figure, curdoc
from bokeh.models.widgets import Slider
from bokeh.layouts import row, widgetbox